## Train a BERT-based model to label semantic equivalence

1. Finetune BERT based on GLOVE dataset
2. BERT -> dropout -> dense output

In [1]:
# Torch 2.0.1
import torch
import numpy as np
from  transformers import BertModel, AutoModelForMaskedLM, BertTokenizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from datasets import load_dataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

c:\Users\liujj\.conda\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Determine token length
MAX_TOKEN_LEN = 256
BATCH_SIZE = 64
EPOCHS = 2


In [3]:
# settings
device = torch.device('cuda')
# load distilbert
model_checkpoint = "bert-base-uncased" #'distilbert-base-uncased'
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
#load dataset
dataset = load_dataset('glue', 'mrpc')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Found cached dataset glue (C:/Users/liujj/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 750.14it/s]


In [4]:
print(dataset['train'][0])

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}


In [5]:
tokenizer.decode(tokenizer.encode(dataset['train'][0]['sentence1'], add_special_tokens=True))

'[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP]'

In [29]:
def bertTokenizeGlue(dataset, tokenizer = tokenizer, max_length = MAX_TOKEN_LEN):

    encoded = tokenizer.encode_plus(
            dataset['sentence1'], dataset['sentence2'],
            add_special_tokens=True, 
            max_length = max_length, 
            padding = 'max_length',
            return_attention_mask = True,
            return_tensors = 'pt'
        )
        
    # convert to pytorch tensors
    labels = dataset['label'] 
    labels = torch.tensor(labels)

    return encoded, labels

def bertTokenizeGlue_all(dataset, tokenizer = tokenizer, max_length = MAX_TOKEN_LEN):
    
    encoded = tokenizer(
            dataset['sentence1'], dataset['sentence2'],
            add_special_tokens=True, 
            max_length = max_length, 
            padding = 'max_length',
            return_attention_mask = True,
            return_tensors = 'pt'
        )
        
    # convert to pytorch tensors
    labels = dataset['label'] 
    labels = torch.tensor(labels)

    return encoded, labels

    
    

In [7]:

class BertDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length):
        super(BertDataset, self).__init__()
        self.dataset = dataset
        self.max_length=max_length
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.dataset.num_rows 
        
    def __getitem__(self, index):
        text = self.dataset[index]
        inputs, labels = bertTokenizeGlue(text, tokenizer=self.tokenizer, max_length=self.max_length)
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        return {
            'input_ids': ids,
            'attention_mask': mask,
            'token_type_ids': token_type_ids,
            'target': labels
            }

In [8]:
class BERTgrader(nn.Module):
    def __init__(self):
        super(BERTgrader, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased") # initialized above
        self.dropout = nn.Dropout(p = 0.2)
        self.outlayer = nn.Linear(768, 1) # output layer
    
    def forward(self, ids, mask, token_type_ids):
        _ , o2 = self.bert(ids,attention_mask=mask,token_type_ids=token_type_ids, return_dict=False)
        o2 = self.dropout(o2)
        out= self.outlayer(o2)
        return out
    


In [14]:
def finetune(epochs,dataloader,grader_model,loss_fn,optimizer):
    grader_model.train()
    for epoch in range(epochs):
        
        loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))
        for batch, dl in loop:
            ids=dl['input_ids'].squeeze()
            token_type_ids=dl['token_type_ids'].squeeze()
            mask= dl['attention_mask'].squeeze()
            label=dl['target']
            label = label.unsqueeze(1)
            optimizer.zero_grad()
            output=grader_model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids)
            label = label.type_as(output)

            loss=loss_fn(output,label)
            loss.backward()
            
            optimizer.step()
            
            pred = np.where(output >= 0, 1, 0)

            num_correct = sum(1 for a, b in zip(pred, label) if a[0] == b[0])
            num_samples = pred.shape[0]
            accuracy = num_correct/num_samples
            
            print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
            
            # Show progress while training
            loop.set_description(f'Epoch={epoch}/{epochs}')
            loop.set_postfix(loss=loss.item(),acc=accuracy)
    

    return grader_model

In [38]:
def scoring(grader_model, test_set, label):
    grader_model.eval()
    output = grader_model(test_set['input_ids'], mask = test_set['attention_mask'], token_type_ids=test_set['token_type_ids'])
    pred = np.where(output >= 0, 1, 0)
    num_correct = sum(1 for a, b in zip(pred, label) if a[0] == b[0])
    num_samples = pred.shape[0]
    accuracy = num_correct/num_samples
    # to add f1 here
    
    print(accuracy)
    

In [17]:
train_data = BertDataset(dataset['train'], tokenizer, max_length=MAX_TOKEN_LEN)
dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE)

In [30]:
valid_x, valid_y = bertTokenizeGlue_all(dataset['validation'], tokenizer, max_length=MAX_TOKEN_LEN)


In [18]:
grader_model = BERTgrader()
loss = nn.BCEWithLogitsLoss()        
optimizer= optim.Adam(model.parameters(),lr= 0.0001)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# a['token_type_ids'].squeeze() # one iter
# grader_model(a['input_ids'].squeeze(), mask=a['attention_mask'].squeeze(),token_type_ids=a['token_type_ids'].squeeze())

In [19]:
grader_model=finetune(EPOCHS, dataloader, grader_model, loss, optimizer)

Epoch=0/2:   2%|▏         | 1/58 [00:50<48:19, 50.86s/it, acc=0.609, loss=0.671]

Got 39 / 64 with accuracy 60.94


Epoch=0/2:   3%|▎         | 2/58 [01:32<42:15, 45.28s/it, acc=0.688, loss=0.643]

Got 44 / 64 with accuracy 68.75


Epoch=0/2:   5%|▌         | 3/58 [02:12<39:15, 42.82s/it, acc=0.828, loss=0.592]

...

Epoch=1/2:  97%|█████████▋| 56/58 [39:46<01:26, 43.39s/it, acc=0.609, loss=0.671]

Got 39 / 64 with accuracy 60.94


Epoch=1/2:  98%|█████████▊| 57/58 [40:27<00:42, 42.57s/it, acc=0.719, loss=0.631]

Got 46 / 64 with accuracy 71.88


Got 15 / 20 with accuracy 75.00


In [41]:
# scoring(grader_model, valid_x, valid_y)